In [1]:
import pandas as pd

In [3]:
oggi = pd.read_csv('20200319-081417.csv')

In [17]:
oggi[oggi['aqi']>'1']

,local date,local time,city,lat,lon,aqi,h,p,pm10,pm25,so2,t,w,wg,co,no2,o3,dew
0,2020-03-17,23:00:00,"Gorizia, Gorizia, Friuli Venezia Giulia, Italy",45.937013,13.616333,74,44.6,1034.8,23.0,74.0,0.1,19.3,1.0,4.2,NaN,NaN,NaN,NaN
1,2020-03-17,23:00:00,"Hera, Emilia Romagna, Italy",44.229397,12.089160,72,58.0,1028.3,27.0,72.0,NaN,14.4,1.0,2.0,0.1,19.7,36.2,NaN
2,2020-03-17,23:00:00,"Como Centro, Lombardia, Italy",45.815042,9.066979,91,29.3,1014.2,37.0,91.0,1.1,6.3,1.1,2.6,0.1,26.6,34.2,NaN
3,2020-03-17,23:00:00,"Bassi, Firenze, Italy",43.784169,11.286880,59,37.6,1024.1,20.0,59.0,0.6,20.5,9.0,16.0,0.1,23.4,NaN,NaN
4,2020-03-19,06:00:00,"Solofra Zona Industriale, Solofra, Campania, I...",40.835550,14.824590,6,1.0,1022.8,30.0,70.0,0.5,0.5,803.0,803.0,5.5,6.4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,2020-03-17,23:00:00,"Pavia Folperti, Lombardia, Italy",45.194682,9.164649,41,81.0,1027.6,32.0,NaN,12.7,7.9,0.5,1.1,NaN,22.0,40.7,NaN
163,2020-03-17,23:00:00,"Ostellato, Emilia Romagna, Italy",44.740896,11.941937,70,59.0,1026.6,15.0,70.0,NaN,18.8,1.0,1.0,0.1,10.1,39.4,NaN
164,2020-03-17,23:00:00,"Staffarda, Revello, Piemonte, Italy",44.720212,7.438307,65,67.2,1015.4,NaN,65.0,NaN,7.7,0.6,1.7,NaN,NaN,NaN,NaN
165,2020-03-17,23:00:00,"Borghetto, Pisa, Italy",43.711537,10.409876,74,43.8,1025.1,28.0,74.0,NaN,19.4,4.0,5.6,0.1,34.3,NaN,NaN
